In [2]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=09bab3b614ca88e8fa3eccd565f97901e6c24d401c1ec24a0e99afb835fde3c0
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [10]:
pip install nltk

In [1]:
from pyspark import SparkContext

In [2]:
sc = SparkContext()

In [3]:
# Print the version of SparkContext
print("The version of Spark Context in the PySpark shell is", sc.version)
# Print the Python version of SparkContext
print("The Python version of Spark Context in the PySpark shell is", sc.pythonVer)
# Print the master of SparkContext
print("The master of Spark Context in the PySpark shell is", sc.master)

The version of Spark Context in the PySpark shell is 3.5.0
The Python version of Spark Context in the PySpark shell is 3.10
The master of Spark Context in the PySpark shell is local[*]


In [16]:
my_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
# Print my_list in the console
print("Input list is", my_list)
# Square all numbers in my_list
squared_list_lambda = list(map(lambda x: x**2, my_list))
# Print the result of the map function
print("The squared numbers are", squared_list_lambda)

Input list is [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
The squared numbers are [1, 4, 9, 16, 25, 36, 49, 64, 81, 100]


In [17]:
my_list2 = [10, 21, 31, 40, 51, 60, 72, 80, 93, 101]
# Print my_list2 in the console
print("Input list is:", my_list2)
# Filter numbers divisible by 10
filtered_list = list(filter(lambda x: (x%10 == 0), my_list2))
# Print the numbers divisible by 10
print("Numbers divisible by 10 are:", filtered_list)

Input list is: [10, 21, 31, 40, 51, 60, 72, 80, 93, 101]
Numbers divisible by 10 are: [10, 40, 60, 80]


In [18]:
# Create an RDD from a list of words
RDD = sc.parallelize(["Spark", "is", "a", "framework", "for", "Big Data processing"])
# Print out the type of the created object
print("The type of RDD is", type(RDD))
# Check the number of partitions in fileRDD
print("Number of partitions in RDD is", RDD.getNumPartitions())

The type of RDD is <class 'pyspark.rdd.RDD'>
Number of partitions in RDD is 2


In [19]:
file_path = "datasets/5000_points.txt"
# Create a fileRDD_part from file_path with 5 partitions
fileRDD = sc.textFile(file_path)
# Check the number of partitions in fileRDD
print("Number of partitions in fileRDD is", fileRDD.getNumPartitions())
# Create a fileRDD_part from file_path with 5 partitions
fileRDD_part = sc.textFile(file_path, minPartitions = 5)
# Check the number of partitions in fileRDD_part
print("Number of partitions in fileRDD_part is", fileRDD_part.getNumPartitions())

Number of partitions in fileRDD is 2
Number of partitions in fileRDD_part is 5


Note that modifying the number of partitions may result in faster performance due to parallelization.

### ReduceBykey and Collect

In [4]:
# Create PairRDD Rdd with key value pairs
Rdd = sc.parallelize([("a",2), ("b",4), ("c",6), ("b",5), ("a", 4)])
# Apply reduceByKey() operation on Rdd
Rdd_Reduced = Rdd.reduceByKey(lambda x, y: x + y)
# Iterate over the result and print the output
for (letter, num) in Rdd_Reduced.collect():
  print("Key {} has {} Counts".format(letter, num))

Key b has 9 Counts
Key c has 6 Counts
Key a has 6 Counts


### SortByKey and Collect

In [5]:
Rdd_Reduced = Rdd_Reduced.map(lambda x : (x[1], x[0]))
# Sort the reduced RDD with the key by descending order
Rdd_Reduced_Sort = Rdd_Reduced.sortByKey(ascending=False)
# Iterate over the result and retrieve all the elements of the RDD
for (num, letter) in Rdd_Reduced_Sort.collect():
  print("Key {} has {} Counts".format(letter, num))

Key b has 9 Counts
Key c has 6 Counts
Key a has 6 Counts


### countByKey

In [7]:
Rdd = sc.parallelize([("a",2), ("b",4), ("c",6), ("b",5), ("a", 4)])
# Count the unique keys
total = Rdd.countByKey()
# What is the type of total?
print("The type of total is", type(total))
# Iterate over the total and print the output
for k, v in total.items():
  print("key", k, "has", v, "counts")

The type of total is <class 'collections.defaultdict'>
key a has 2 counts
key b has 2 counts
key c has 1 counts


### Calculates the most common words from Complete Works of William Shakespeare.

In [9]:
file_path = "datasets/Complete_Shakespeare.txt"
# Create a baseRDD from the file path
baseRDD = sc.textFile(file_path)
# Split the lines of baseRDD into words
splitRDD = baseRDD.flatMap(lambda x: x.split())
# Count the total number of words
print("Total number of words in splitRDD:", splitRDD.count())

Total number of words in splitRDD: 128576


In [11]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [13]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [14]:
# Convert the words in lower case and remove stop words from the stop_words curated list
splitRDD_no_stop = splitRDD.filter(lambda x: x.lower() not in stop_words)
# Create a tuple of the word and 1
splitRDD_no_stop_words = splitRDD_no_stop.map(lambda w: (w, 1))
# Count of the number of occurences of each word
resultRDD = splitRDD_no_stop_words.reduceByKey(lambda x, y: x + y)

In [15]:
# Display the first 10 words and their frequencies from the input RDD
for word in resultRDD.take(10):
	print(word)
# Swap the keys and values from the input RDD
resultRDD_swap = resultRDD.map(lambda x: (x[1], x[0]))
# Sort the keys in descending order
resultRDD_swap_sort = resultRDD_swap.sortByKey(ascending=False)
# Show the top 10 most frequent words and their frequencies from the sorted RDD
for word in resultRDD_swap_sort.take(10):
	print("{},{}". format(word[1], word[0]))

('Project', 9)
('EBook', 1)
('Shakespeare', 12)
('use', 38)
('anyone', 1)
('anywhere', 1)
('restrictions', 1)
('whatsoever.', 1)
('may', 162)
('it,', 74)
thou,650
thy,574
shall,393
would,311
good,295
thee,286
love,273
Enter,269
th',254
make,225
